In [1]:
import pandas as pd
import numpy as np
import wikipedia as wp
import requests
import re
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
#Get the html source
html = wp.page("List of United States cities by population").html().encode("UTF-8")

df = pd.read_html(html)
df = df[4] # choose this table specifically

# change column names
df.columns = df.iloc[0] 
df = df.reindex(df.index.drop(0)) 

df = df.loc[:, df.columns.notnull()] # delete useless columns

df = df.rename(index=str, columns={"State[c]": "State"}) # rename columns

# rename cities when there is a format error
for index,i in enumerate(df.City):
    if i[-1] == "]":
        i = i[:-3]
        df.City[index] = i

In [3]:
df.head()

,2018rank,City,State,2018estimate,2010Census,Change,2016 land area,2016 population density,Location
1,1,New York City,New York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi"
2,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi"
3,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi"
4,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi"
5,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi"


In [52]:
to_append = []
for ind,i in tqdm(enumerate(df.City)):
    
    # get the web url
    try:
        page = wp.page(i).url
        #website_url = requests.get(page).text
    except: # if there is an ambiguity problem
        try:
            page = wp.page(i+", "+df.State[ind]).url
            #website_url = requests.get(page).text
        except:
            print(i+", "+df.State[ind])
            to_append.append("??")
            continue
            
    #page = requests.get(website_url)
    #soup = BeautifulSoup(page.content, 'html.parser')
    soup = BeautifulSoup(requests.get(page).content, 'html.parser')
    
    try:
        to_add = soup.find("a", href=re.compile("/wiki/Time_zone")).find_next("td").text
        to_append.append(to_add)
    except:
        to_append.append("Not disclosed")

155it [03:39,  1.09s/it]

Cary, North Carolina


311it [07:32,  1.10s/it]

Woodbridge[, New Jersey


314it [07:36,  1.37s/it]


In [54]:
to_append[154] = "UTC-5 (EST)"
to_append[310] = "UTC−5 (Eastern (EST))"

In [57]:
df['Time_zone'] = to_append

In [63]:
for index,i in enumerate(df.City):
    if i[-1] == "[":
        i = i[:-1]
        df.City[index] = i

In [64]:
df

,2018rank,City,State,2018estimate,2010Census,Change,2016 land area,2016 population density,Location,Time_zone
1,1,New York City,New York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi",UTC−05:00 (EST)
2,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi",UTC−08:00 (Pacific)
3,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi",UTC−06:00 (Central)
4,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi",UTC−6 (CST)
5,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi",UTC−7 (MST (no DST))
6,6,Philadelphia,Pennsylvania,1584138,1526006,+3.81%,134.2 sq mi,347.6 km2,"11,683/sq mi",UTC-5 (EST)
7,7,San Antonio,Texas,1532233,1327407,+15.43%,461.0 sq mi,"1,194.0 km2","3,238/sq mi",UTC−6 (CST)
8,8,San Diego,California,1425976,1307402,+9.07%,325.2 sq mi,842.3 km2,"4,325/sq mi",UTC−8 (Pacific)
9,9,Dallas,Texas,1345047,1197816,+12.29%,340.9 sq mi,882.9 km2,"3,866/sq mi",UTC−6 (Central)
10,10,San Jose,California,1030119,945942,+8.90%,177.5 sq mi,459.7 km2,"5,777/sq mi",UTC−8 (Pacific Time Zone)


In [65]:
df.to_csv("final.csv")